In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
import logging
import glob
import random
import pydicom
import json
import PIL
import boto3

from tqdm import tqdm_notebook
from multiprocessing import Pool
from multiprocessing import cpu_count

sys.path.append('../')
from rsna import utils

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
! ls /mirco-kaggle/rsna/

all.zip				     stage_1_test_images_dcm_dev
GCP Credits Request Link - RSNA.txt  stage_1_test_images_jpg_dev
label_map.pbtxt			     stage_1_test_images.zip
stage_1_detailed_class_info.csv      stage_1_train_images
stage_1_sample_submission.csv	     stage_1_train_images.zip
stage_1_test_images		     stage_1_train_labels.csv


In [66]:
source_images_path = "/mirco-kaggle/rsna/stage_1_test_images/"

In [65]:
# destination_images_path = "/mirco-kaggle/rsna/stage_1_test_images_jpg_dev/"
destination_images_path = "/home/mhess/stage_1_test_images_jpg_dev/"

In [67]:
dcm_image_filepaths = glob.glob(os.path.join(source_images_path, "*.dcm"))

In [68]:
len(dcm_image_filepaths)

1000

In [69]:
def dcm_to_jpg(dcm_image_filepath):
    
    image_filename = os.path.basename(dcm_image_filepath).split(".")[0]
    
    image_array = utils.Annot._read_dcm(dcm_image_filepath)
    image_pil = PIL.Image.fromarray(image_array)
    image_pil.save(os.path.join(destination_images_path, image_filename + ".jpg"), "JPEG", quality=100)
#     print(image_filename)

In [70]:
with Pool(cpu_count()) as p:
    tf_examples = list(tqdm_notebook(p.imap(dcm_to_jpg, dcm_image_filepaths), total=len(destination_images_path)))

In [51]:
destination_images_path = "/home/mhess/stage_1_test_images_jpg_dev/"

In [63]:

def change():
    path_images = destination_images_path
    filenames_src = tf.gfile.ListDirectory(path_images)
    for filename_src in tqdm_notebook(filenames_src):
        stem, extension = os.path.splitext(filename_src)
        if (extension.lower() != '.jpg'): continue


        pathname_jpg = '{}/{}'.format(path_images, filename_src)
        with tf.gfile.GFile(pathname_jpg, 'rb') as fid:
            encoded_jpg = fid.read(4)
        # png
        if(encoded_jpg[0] == 0x89 and encoded_jpg[1] == 0x50 and encoded_jpg[2] == 0x4e and encoded_jpg[3] == 0x47):
            # copy jpg->png then encode png->jpg
            print('png:{}'.format(filename_src))
            pathname_png = '{}/{}.png'.format(path_images, stem)
            tf.gfile.Copy(pathname_jpg, pathname_png, True)
            PIL.Image.open(pathname_png).convert('RGB').save(pathname_jpg, "jpeg")   
        # gif
        elif(encoded_jpg[0] == 0x47 and encoded_jpg[1] == 0x49 and encoded_jpg[2] == 0x46):
            # copy jpg->gif then encode gif->jpg
            print('gif:{}'.format(filename_src))
            pathname_gif = '{}/{}.gif'.format(path_images, stem)
            tf.gfile.Copy(pathname_jpg, pathname_gif, True)
            PIL.Image.open(pathname_gif).convert('RGB').save(pathname_jpg, "jpeg")   
        elif(filename_src == 'beagle_116.jpg' or filename_src == 'chihuahua_121.jpg'):
            # copy jpg->jpeg then encode jpeg->jpg
            print('jpeg:{}'.format(filename_src))
            pathname_jpeg = '{}/{}.jpeg'.format(path_images, stem)
            tf.gfile.Copy(pathname_jpg, pathname_jpeg, True)
            PIL.Image.open(pathname_jpeg).convert('RGB').save(pathname_jpg, "jpeg")   
        elif(encoded_jpg[0] != 0xff or encoded_jpg[1] != 0xd8 or encoded_jpg[2] != 0xff):
            print('not jpg:{}'.format(filename_src))


In [64]:
change()